# TP1 - Exercício 1
## Grupo 1

*   Diogo Coelho da Silva A100092
*   Pedro Miguel Ramôa Oliveira A97686

**Problema proposto:**

1.  Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:
    - **a)** Cada reunião ocupa uma sala (enumeradas 1...S) durante um “slot” (tempo,dia).  Assume-se os dias enumerados 1..D e, em cada dia, os tempos enumerados 1..T.
    - **b)** Cada reunião tem associado um projeto (enumerados 1..P) e um conjunto de participantes. Os diferentes colaboradores são enumerados 1..C.
    - **c)** Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. São “inputs” do problema o conjunto de colaboradores de cada projeto, o seu líder e o número de reuniões semanais.
    - **d)** O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  50% do total de colaboradores do projeto.  A disponibilidade de cada participante, incluindo o lider,  é um conjunto de “slots” (“inputs” do problema).

**Proposta de resolução:**

O problema que foi apresentado tem como objetivo a criação de um horário semanal otimizado para a marcação de reuniões para uma *startup*. O objetivo é alocar eficientemente recursos como salas e tempo, considerando a disponibilidade dos colaboradores e as restrições dadas no enunciado.

Na solução proposta iremos modelar o problema de acordo com as nossas necessidades e utilizaremos um solver SCIP para encontrar uma solução otimizada.

Foram consideradas restrições dadas pelo enunciado, como por exemplo, a obrigatoriedade da presença do lider em todas as reuniões do seu projeto, a disponibilidade dos colaboradores, a partição exclusiva dos membros do projeto, a alocação de apenas um projeto por sala em cada slot e a garantia de um quorum minimo em cada reunião.









#### 1. Importar as bibliotecas importantes

In [1]:
from ortools.linear_solver import pywraplp
import random
import pandas as pd

- **pywraplp**: Importa o solver de programação linear da biblioteca OR-Tools para resolver o problema de otimização.
- **random**: Gera aleatoriedade para simular a disponibilidade de colaboradores e equipes dos projetos.
- **pandas**: Biblioteca usada para organizar os dados em formato de tabela (DataFrame), facilitando a manipulação e visualização.

In [ ]:


# Parâmetros do problema
S = 5  # Número de salas
D = 5  # Número de dias
H = 8  # Número de horas (slots por dia)
P = 5  # Número de projetos
C = 30 # Número de colaboradores

# Definir slots e colaboradores
Slots = [(d, h) for d in range(D) for h in range(H)]
Colabs = set(range(C))

# Definir disponibilidade dos colaboradores aleatoriamente
random.seed(42)  # Definir a seed para reprodução dos resultados
Colaboradores = [random.sample(Slots, 20) for _ in range(C)]

# Definir projetos e seus times (líder + colaboradores)
Projectos = []
for _ in range(P):
    team = random.sample(list(Colabs), 6)
    Projectos.append((random.randint(1, 5), team))  # (num_reunioes_semanais, [lista_de_colaboradores])
    Colabs = Colabs - set(team)

# Inicializando o solver SCIP
horario = pywraplp.Solver.CreateSolver('SCIP')

# Variáveis de decisão: x[s, d, h, p, c] = 1 se o colaborador c participa na reunião do projeto p no slot (sala s, dia d, hora h)
x = {}
for s in range(S):
    for d in range(D):
        for h in range(H):
            for p in range(P):
                for c in range(C):
                    x[s, d, h, p, c] = horario.BoolVar(f'x[{s},{d},{h},{p},{c}]')

# Restrição 1: O líder tem de estar em todas as R reuniões do seu projecto
for p in range(P):
    lider = Projectos[p][1][0]
    horario.Add(
        sum(x[s, d, h, p, lider] for s in range(S) for d in range(D) for h in range(H)) == Projectos[p][0]
    )

# Restrição 2: Colaboradores fora da disponibilidade não podem participar
for s in range(S):
    for d in range(D):
        for h in range(H):
            for p in range(P):
                for c in range(C):
                    if (d, h) not in Colaboradores[c]:
                        horario.Add(x[s, d, h, p, c] == 0)

# Restrição 3: Apenas colaboradores do projeto podem participar
for s in range(S):
    for d in range(D):
        for h in range(H):
            for p in range(P):
                for c in range(C):
                    if c not in Projectos[p][1]:
                        horario.Add(x[s, d, h, p, c] == 0)

# Restrição 4: Uma sala pode ter apenas um projeto por slot de tempo
for s in range(S):
    for d in range(D):
        for h in range(H):
            horario.Add(sum(x[s, d, h, p, Projectos[p][1][0]] for p in range(P)) <= 1)

# Restrição 5: Um colaborador só pode estar numa sala em um dado slot
for d in range(D):
    for h in range(H):
        for p in range(P):
            for c in Projectos[p][1]:
                horario.Add(sum(x[s, d, h, p, c] for s in range(S)) <= 1)

# Restrição 6: Quorum de 50% dos colaboradores, incluindo o líder
for s in range(S):
    for d in range(D):
        for h in range(H):
            for p in range(P):
                quorum = len(Projectos[p][1]) // 2 + 1  # 50% ou mais com o líder
                horario.Add(sum(x[s, d, h, p, c] for c in Projectos[p][1]) >= quorum * x[s, d, h, p, Projectos[p][1][0]])

# Função objetivo: Maximizar o uso de slots para reuniões
horario.Maximize(
    sum(x[s, d, h, p, c] for s in range(S) for d in range(D) for h in range(H) for p in range(P) for c in range(C))
)

# Resolver o problema
status = horario.Solve()

# Mapeamento dos slots para horas
time_slots = ["08:30 - 09:30", "09:30 - 10:30", "10:30 - 11:30", "11:30 - 12:30",
              "13:30 - 14:30", "14:30 - 15:30", "15:30 - 16:30", "16:30 - 17:30"]

# Verificar a solução e preparar a tabela de resultados
schedule_data = []

if status == pywraplp.Solver.OPTIMAL:
    for p in range(P):
        for s in range(S):
            for d in range(D):
                for h in range(H):
                    if round(x[s, d, h, p, Projectos[p][1][0]].solution_value()) == 1:
                        participants = [c for c in Projectos[p][1] if round(x[s, d, h, p, c].solution_value()) == 1]
                        schedule_data.append([p + 1, s + 1, d, time_slots[h], participants])
else:
    schedule_data.append(["Nenhuma solução ótima encontrada."])


# Criar dataframe da tabela
df_schedule = pd.DataFrame(schedule_data, columns=["Projeto", "Sala", "Dia", "Hora", "Participantes"])

# Criar estilo para linhas alternadas e bordas entre colunas
def style_schedule(df):
    return df.style.set_table_styles(
        [{'selector': 'tr:nth-child(even)',
          'props': [('background-color', '#f2f2f2')]},   # Cor alternada para linhas pares
         {'selector': 'th, td',
          'props': [('border', '1px solid black')]}      # Bordas em volta das células
        ]
    ).set_properties(**{'text-align': 'center'})          # Alinhar o texto no centro

# Aplicar estilo e exibir diretamente no notebook
styled_table = style_schedule(df_schedule)

# Exibir a tabela estilizada no Jupyter Notebook
display(styled_table)



,Projeto,Sala,Dia,Hora,Participantes
0,1,1,1,10:30 - 11:30,"[23, 27, 14, 8]"
1,1,1,4,10:30 - 11:30,"[23, 15, 14, 24]"
2,2,1,2,14:30 - 15:30,"[25, 19, 18, 7]"
3,2,1,3,14:30 - 15:30,"[25, 19, 18, 28]"
4,2,1,4,14:30 - 15:30,"[25, 19, 7, 28]"
5,2,2,2,09:30 - 10:30,"[25, 9, 18, 7]"
6,3,1,0,09:30 - 10:30,"[2, 12, 10, 4, 20]"
7,3,1,2,13:30 - 14:30,"[2, 10, 4, 5]"
8,3,1,3,16:30 - 17:30,"[2, 12, 5, 20]"
9,3,1,4,11:30 - 12:30,"[2, 10, 4, 20]"
